In [1]:
pip install pyproj scikit-learn pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from pyproj import Transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

# Пример данных о локациях
data = {
    'latitude': [40.7128, 40.7306, 40.7484, 40.7580, 40.7688, 40.7788, 40.7888, 40.7988],
    'longitude': [-74.0060, -73.9352, -73.9857, -73.9857, -73.9681, -73.9581, -73.9481, -73.9381],
    'idealness': [0.8, 0.9, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]  # Примерные оценки идеальности
}

df = pd.DataFrame(data)

# Создаем трансформер для проекции Меркатора
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

# Преобразуем географические координаты в плоские
df['x'], df['y'] = transformer.transform(df['longitude'].values, df['latitude'].values)

# Функция для расчета манхэттенского расстояния
def manhattan_distance(x1, y1, x2, y2):
    return abs(x2 - x1) + abs(y2 - y1)

# Расчет матрицы манхэттенских расстояний
def compute_manhattan_distances(df):
    n = len(df)
    distances = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            distances[i, j] = manhattan_distance(df.loc[i, 'x'], df.loc[i, 'y'], df.loc[j, 'x'], df.loc[j, 'y'])
    return distances

# Вычисляем матрицу манхэттенских расстояний
distances_matrix = compute_manhattan_distances(df)

# Добавляем расстояния до всех локаций в качестве признаков
distances_df = pd.DataFrame(distances_matrix, columns=[f'distance_to_{j}' for j in range(len(df))])
df = pd.concat([df, distances_df], axis=1)

# Разделяем данные на обучающую и тестовую выборки
X = df.drop(columns=['latitude', 'longitude', 'idealness'])
y = df['idealness']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. Обучение модели ансамбля деревьев (RandomForestRegressor)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Делаем предсказания на тестовой выборке
y_pred_rf = rf_model.predict(X_test)

# Оцениваем качество модели
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Среднеквадратичная ошибка для RandomForest: {mse_rf}")

# 2. Обучение модели CatBoost
catboost_model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, random_state=42, verbose=0)
catboost_model.fit(X_train, y_train)

# Делаем предсказания на тестовой выборке
y_pred_catboost = catboost_model.predict(X_test)

# Оцениваем качество модели
mse_catboost = mean_squared_error(y_test, y_pred_catboost)
print(f"Среднеквадратичная ошибка для CatBoost: {mse_catboost}")

# 3. Кросс-валидация в CatBoost
cv_scores = cross_val_score(catboost_model, X, y, cv=5, scoring='neg_mean_squared_error')
mse_cv = -cv_scores.mean()
print(f"Среднеквадратичная ошибка после кросс-валидации: {mse_cv}")

# Предсказываем идеальность для новых локаций
new_locations = pd.DataFrame({
    'latitude': [40.7530, 40.7688],
    'longitude': [-73.9824, -73.9681]
})

new_locations['x'], new_locations['y'] = transformer.transform(new_locations['longitude'].values, new_locations['latitude'].values)

# Вычисляем матрицу манхэттенских расстояний для новых локаций
new_distances_matrix = np.zeros((len(new_locations), len(df)))
for i in range(len(new_locations)):
    for j in range(len(df)):
        new_distances_matrix[i, j] = manhattan_distance(new_locations.loc[i, 'x'], new_locations.loc[i, 'y'], df.loc[j, 'x'], df.loc[j, 'y'])

# Добавляем расстояния до всех локаций в качестве признаков
new_distances_df = pd.DataFrame(new_distances_matrix, columns=[f'distance_to_{j}' for j in range(len(df))])
new_locations = pd.concat([new_locations, new_distances_df], axis=1)

# Предсказания с использованием RandomForest
new_locations_pred_rf = rf_model.predict(new_locations.drop(columns=['latitude', 'longitude']))
print(f"Предсказанная идеальность для новых локаций с использованием RandomForest: {new_locations_pred_rf}")

# Предсказания с использованием CatBoost
new_locations_pred_catboost = catboost_model.predict(new_locations.drop(columns=['latitude', 'longitude']))
print(f"Предсказанная идеальность для новых локаций с использованием CatBoost: {new_locations_pred_catboost}")

Среднеквадратичная ошибка для RandomForest: 0.06018049999999991
Среднеквадратичная ошибка для CatBoost: 0.03849639418557779
Среднеквадратичная ошибка после кросс-валидации: 0.033857508666296735
Предсказанная идеальность для новых локаций с использованием RandomForest: [0.619 0.502]
Предсказанная идеальность для новых локаций с использованием CatBoost: [0.61044892 0.50165123]
